In [30]:
import numpy as np

In [31]:
import idx2numpy

y_train = idx2numpy.convert_from_file('../dataset/train-labels.idx1-ubyte')
y_test = idx2numpy.convert_from_file('../dataset/t10k-labels.idx1-ubyte')

In [32]:
text = ''

In [54]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [33]:
for x in y_train:
    text+=str(x)

In [34]:
for x in y_test:
    text+=str(x)

In [35]:
print (len(text))
print (text[0:9])

70000
504192131


In [36]:
# Length of extracted character sequences
maxlen = 50

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x_data = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_data = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_data[i, t, char_indices[char]] = 1
    y_data[i, char_indices[next_chars[i]]] = 1

Number of sequences: 23317
Unique characters: 10
Vectorization...


In [37]:
from keras import layers, models, optimizers

model = models.Sequential()
model.add(layers.LSTM(256, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [38]:
optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [39]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 504

In [62]:
x_data[0]

array([[False, False, False, False, False,  True, False, False, False,
        False],
       [ True, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False,  True, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
         True],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False, False, False,  True, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False, False, False, False,  True, False, False, False, False,
        False],
       [False, False, False,  True, False, False, False, False, False,
        False],
       [False, False, False, False, False, 

# 3

In [63]:
y_data[0]

array([False, False, False,  True, False, False, False, False, False, False], dtype=bool)

In [40]:
# Fit the model for 1 epoch on the available training data
model.fit(x_data, y_data, batch_size=128, epochs=1)

Epoch 1/1
23317/23317 [==============================] - 28s 1ms/step - loss: 2.3277


In [64]:
test_to_perform

3

In [41]:
import random
import sys

chars_to_generate = 50
test_to_perform = 3

seed_text = []
text_to_generate = []

for epoch in range(test_to_perform):

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    end_index = start_index + maxlen
    
    seed_text.append(text[start_index: end_index])
    text_to_generate.append(text[end_index: end_index + chars_to_generate])

    
    print('Seed Text {}: "'.format(epoch) + str(seed_text[epoch]) + '"')
    print('OG Text   {}: "'.format(epoch) + str(text_to_generate[epoch]) + '"')       
    print()
    #model.save('model_epoch_{}.hdf5'.format(epoch))
    #model.save_weights('text_generator_gigantic_weights{}.h5'.format(epoch))

Seed Text 0: "83500578895168117690604152835435066708890001120394"
OG Text   0: "15665790914283644516278879364432969487252557086922"

Seed Text 1: "38466279513243619447654199278013613411156070723252"
OG Text   1: "29498121612780008229227992751349418562831284993707"

Seed Text 2: "67682409991711497873842826338856904110066293182960"
OG Text   2: "97128314351592789514562696377961283241228695731477"



In [68]:
seed_text

['83500578895168117690604152835435066708890001120394',
 '38466279513243619447654199278013613411156070723252',
 '67682409991711497873842826338856904110066293182960']

In [79]:
text_generated_0_2 = []
text_generated_0_5 = []
text_generated_1_0 = []
text_generated_1_2 = []

for x in range(len(seed_text)):
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        gentext = ''
        # We generate 50 characters
        for i in range(chars_to_generate):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(seed_text[x]):
                sampled[0, t, char_indices[char]] = 1.
            print("sampled: ",len(sampled[0]))
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            gentext += '{}'.format(next_char)
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
        if temperature == 0.2:
            text_generated_0_2.append(gentext)
        if temperature == 0.5:
            text_generated_0_5.append(gentext)
        if temperature == 1.0:
            text_generated_1_0.append(gentext)
        if temperature == 1.2:
            text_generated_1_2.append(gentext)

------ temperature: 0.2
sampled:  50
0sampled:  50
8sampled:  50
1sampled:  50
8sampled:  50
9sampled:  50
6sampled:  50
6sampled:  50
1sampled:  50
4sampled:  50
1sampled:  50
8sampled:  50
1sampled:  50
8sampled:  50
1sampled:  50
7sampled:  50
1sampled:  50
1sampled:  50
8sampled:  50
1sampled:  50
1sampled:  50
6sampled:  50
8sampled:  50
6sampled:  50
8sampled:  50
8sampled:  50
6sampled:  50
6sampled:  50
6sampled:  50
3sampled:  50
8sampled:  50
9sampled:  50
8sampled:  50
1sampled:  50
8sampled:  50
8sampled:  50
6sampled:  50
1sampled:  50
8sampled:  50
8sampled:  50
1sampled:  50
6sampled:  50
8sampled:  50
7sampled:  50
3sampled:  50
0sampled:  50
8sampled:  50
1sampled:  50
1sampled:  50
6sampled:  50
8
------ temperature: 0.5
sampled:  50
4sampled:  50
9sampled:  50
4sampled:  50
1sampled:  50
8sampled:  50
3sampled:  50
8sampled:  50
1sampled:  50
0sampled:  50
0sampled:  50
4sampled:  50
3sampled:  50
5sampled:  50
4sampled:  50
3sampled:  50
7sampled:  50
6sampled:  50


2sampled:  50
4sampled:  50
5sampled:  50
2sampled:  50
5sampled:  50
1sampled:  50
9sampled:  50
5sampled:  50
2sampled:  50
7sampled:  50
8sampled:  50
5sampled:  50
5sampled:  50
9sampled:  50
5sampled:  50
4sampled:  50
0sampled:  50
5sampled:  50
5sampled:  50
4sampled:  50
8sampled:  50
2sampled:  50
7sampled:  50
5sampled:  50
4sampled:  50
5sampled:  50
9sampled:  50
9sampled:  50
6sampled:  50
0sampled:  50
9sampled:  50
4sampled:  50
4sampled:  50
6sampled:  50
0sampled:  50
4sampled:  50
3


In [43]:
for x in range(test_to_perform):
    print ('{} seed text:            '.format(x), seed_text[x])
    print ('{} text to be generated: '.format(x), text_to_generate[x])
    print ('{} text generated 0.2:   '.format(x), text_generated_0_2[x])
    print ('{} text generated 0.5:   '.format(x), text_generated_0_5[x])
    print ('{} text generated 1.0:   '.format(x), text_generated_1_0[x])
    print ('{} text generated 1.2:   '.format(x), text_generated_1_2[x])
    print ()

0 seed text:             83500578895168117690604152835435066708890001120394
0 text to be generated:  15665790914283644516278879364432969487252557086922
0 text generated 0.2:    44444444444444444444444444444444744444444444444444
0 text generated 0.5:    44480874445454774777845444744744444444744447454754
0 text generated 1.0:    78742516054847893455748785589874444454914451754743
0 text generated 1.2:    37412514474165448355079787444047525977105458157148

1 seed text:             38466279513243619447654199278013613411156070723252
1 text to be generated:  29498121612780008229227992751349418562831284993707
1 text generated 0.2:    44444844474744774747474744744474444444744447444447
1 text generated 0.5:    77344890417474687447447477847174543444474444744434
1 text generated 1.0:    71778739770535773326174489807037971440393471487943
1 text generated 1.2:    47404373944836885848806475461014259547367564431826

2 seed text:             67682409991711497873842826338856904110066293182960
2 text to 

In [44]:
count_0_2 = []
per_0_2 = []
count_0_5 = []
per_0_5 = []
count_1_0 = []
per_1_0 = []
count_1_2 = []
per_1_2 = []

for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_0_2[y][x]:
            a+=1
    count_0_2.append(a)
    per_0_2.append((a/chars_to_generate)*100)


for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_0_5[y][x]:
            a+=1
    count_0_5.append(a)
    per_0_5.append((a/chars_to_generate)*100)

    
for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_1_0[y][x]:
            a+=1
    count_1_0.append(a)
    per_1_0.append((a/chars_to_generate)*100)

    
for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_1_2[y][x]:
            a+=1
    count_1_2.append(a)
    per_1_2.append((a/chars_to_generate)*100)

print()
print ('temp 0.2')
for x in range(test_to_perform):
    print (count_0_2[x], per_0_2[x])
    
print()
print ('temp 0.5')
for x in range(test_to_perform):
    print (count_0_5[x], per_0_5[x])

print()
print ('temp 1.0')
for x in range(test_to_perform):
    print (count_1_0[x], per_1_0[x])

print()
print ('temp 1.2')
for x in range(test_to_perform):
    print (count_1_2[x], per_1_2[x])



temp 0.2
6 12.0
5 10.0
7 14.000000000000002

temp 0.5
4 8.0
3 6.0
4 8.0

temp 1.0
4 8.0
2 4.0
5 10.0

temp 1.2
6 12.0
4 8.0
10 20.0


## Additional Training

In [45]:
# Fit the model for more 10 epoch on the data
model.fit(x_data, y_data, batch_size=128, epochs=9)

Epoch 1/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.2304
Epoch 2/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.1428
Epoch 3/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.1084
Epoch 4/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.0894
Epoch 5/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.0701
Epoch 6/9
23317/23317 [==============================] - 29s 1ms/step - loss: 2.0341
Epoch 7/9
23317/23317 [==============================] - 29s 1ms/step - loss: 1.9725
Epoch 8/9
23317/23317 [==============================] - 29s 1ms/step - loss: 1.8791
Epoch 9/9
23317/23317 [==============================] - 29s 1ms/step - loss: 1.7628


In [46]:
text_generated_0_2 = []
text_generated_0_5 = []
text_generated_1_0 = []
text_generated_1_2 = []

for x in range(len(seed_text)):
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        gentext = ''
        # We generate 50 characters
        for i in range(chars_to_generate):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(seed_text[x]):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            gentext += '{}'.format(next_char)
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
        if temperature == 0.2:
            text_generated_0_2.append(gentext)
        if temperature == 0.5:
            text_generated_0_5.append(gentext)
        if temperature == 1.0:
            text_generated_1_0.append(gentext)
        if temperature == 1.2:
            text_generated_1_2.append(gentext)

------ temperature: 0.2
18816983838819614183318193169663869863866816618111
------ temperature: 0.5
18684298877397767188217371911317226181868660131888
------ temperature: 1.0
41340363980311873360954171078457194744330891603931
------ temperature: 1.2
70925959106801688741430814461793365560618127676873
------ temperature: 0.2
88888888888888888889888588888888888989888889848888
------ temperature: 0.5
88888880888889804898888889808981888888889988886988
------ temperature: 1.0
90597888984908844051469889689085889448391417789890
------ temperature: 1.2
94284598671375449847681064583708994870884820105648
------ temperature: 0.2
11555555955559545055555555055455559945551559455015
------ temperature: 0.5
90059959511595990515595094955542454525934544401914
------ temperature: 1.0
01359905241348984614691562092525524659550407355470
------ temperature: 1.2
49964433460715463294333604532400750444249645314644


In [80]:
seed_text

['83500578895168117690604152835435066708890001120394',
 '38466279513243619447654199278013613411156070723252',
 '67682409991711497873842826338856904110066293182960']

In [69]:
text_generated_0_2

['18816983838819614183318193169663869863866816618111',
 '88888888888888888889888588888888888989888889848888',
 '11555555955559545055555555055455559945551559455015']

In [70]:
text_generated_0_5

['18684298877397767188217371911317226181868660131888',
 '88888880888889804898888889808981888888889988886988',
 '90059959511595990515595094955542454525934544401914']

In [71]:
text_generated_1_0

['41340363980311873360954171078457194744330891603931',
 '90597888984908844051469889689085889448391417789890',
 '01359905241348984614691562092525524659550407355470']

In [72]:
text_generated_1_2

['70925959106801688741430814461793365560618127676873',
 '94284598671375449847681064583708994870884820105648',
 '49964433460715463294333604532400750444249645314644']

In [49]:
for x in range(test_to_perform,test_to_perform+8):
    print(x)
    print ('{} seed text:            '.format(x), seed_text[x])
    print ('{} text to be generated: '.format(x), text_to_generate[x])
    print ('{} text generated 0.2:   '.format(x), text_generated_0_2[x])
    print ('{} text generated 0.5:   '.format(x), text_generated_0_5[x])
    print ('{} text generated 1.0:   '.format(x), text_generated_1_0[x])
    print ('{} text generated 1.2:   '.format(x), text_generated_1_2[x])
    print ()

3


IndexError: list index out of range

In [ ]:
count_0_2 = []
per_0_2 = []
count_0_5 = []
per_0_5 = []
count_1_0 = []
per_1_0 = []
count_1_2 = []
per_1_2 = []

for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_0_2[y][x]:
            a+=1
    count_0_2.append(a)
    per_0_2.append((a/chars_to_generate)*100)


for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_0_5[y][x]:
            a+=1
    count_0_5.append(a)
    per_0_5.append((a/chars_to_generate)*100)

    
for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_1_0[y][x]:
            a+=1
    count_1_0.append(a)
    per_1_0.append((a/chars_to_generate)*100)

    
for y in range(test_to_perform):
    a=0
    for x in range(chars_to_generate):
        if text_to_generate[y][x] == text_generated_1_2[y][x]:
            a+=1
    count_1_2.append(a)
    per_1_2.append((a/chars_to_generate)*100)

print()
print ('temp 0.2')
for x in range(test_to_perform):
    print (count_0_2[x], per_0_2[x])
    
print()
print ('temp 0.5')
for x in range(test_to_perform):
    print (count_0_5[x], per_0_5[x])

print()
print ('temp 1.0')
for x in range(test_to_perform):
    print (count_1_0[x], per_1_0[x])

print()
print ('temp 1.2')
for x in range(test_to_perform):
    print (count_1_2[x], per_1_2[x])
